## The 0-1 programming model for the shortest path problem with turn restrictions (SPPTR)

In [226]:
# Transform the network
def transform(network, tr, s, d):
    edges = []
    E = []
    w = []
    T = []
    S = []
    D = []
    E_ind = {}
    ind = 0
    for i in network.keys():
        for j in network[i].keys():
            E_ind[str((i, j))] = ind
            E.append(ind)
            edges.append([i, j])
            w.append(network[i][j])
            if i == s:
                S.append(ind)
            if j == d:
                D.append(ind)
            ind += 1
    
    FE = [[] for _ in range(ind)]
    BE = [[] for _ in range(ind)]
    
    for t in tr:
        ind1 = E_ind[str((t[0], t[1]))]
        ind2 = E_ind[str((t[1], t[2]))]
        T.append([ind1, ind2])
    
    for i in network.keys():
        for j in network[i].keys():
            ind1 = E_ind[str((i, j))]
            for k in network[j]:
                ind2 = E_ind[str((j, k))]
                if [ind1, ind2] not in T:
                    FE[ind1].append(ind2)
                    BE[ind2].append(ind1)

    return E, w, S, D, FE, BE, edges

In [227]:
# Input the network
test_network = {
        0: {2: 1},
        1: {2: 1},
        2: {0: 1, 1: 1, 3: 2, 5: 1},
        3: {4: 1},
        4: {5: 2},
        5: {2: 1},
    }
s = 0
d = 5
tr = [[0, 2, 5]]
E, w, S, D, FE, BE, edges = transform(test_network, tr, s, d)

In [228]:
# Create optimization model
from docplex.mp.model import Model
spptr_model = Model(name='spptr')

In [229]:
# Define decision variables
ne = len(E)  # the number of edges
x = spptr_model.binary_var_list(ne, name='x')
u = spptr_model.binary_var_matrix(ne, ne, name='u')

In [230]:
# Define constraints
# Eq.(1)
eq1 = (sum(u[i, j] for j in E if j not in FE[i]) == 0 for i in E)
c1 = spptr_model.add_constraints(eq1, names='eq1')

# Eq.(2)
# eq2 = (sum(u[i, j] for j in FE[i]) == 0 for i in D if FE[i])
eq2 = (sum(u[i, j] for j in FE[i]) == 0 for i in D)
c2 = spptr_model.add_constraints(eq2, names='eq2')

# Eq.(3)
# eq3 = (sum(u[i, j] for j in FE[i]) == x[i] for i in E if i not in D and FE[i])
eq3 = (sum(u[i, j] for j in FE[i]) == x[i] for i in E if i not in D)
c3 = spptr_model.add_constraints(eq3, names='eq3')

# Eq.(4)
# eq4 = (sum(u[i, j] for i in BE[j]) == 0 for j in S if BE[j])
eq4 = (sum(u[i, j] for i in BE[j]) == 0 for j in S)
c4 = spptr_model.add_constraints(eq4, names='eq4')

# Eq.(5)
# eq5 = (sum(u[i, j] for i in BE[j]) == x[j] for j in E if j not in S and BE[j])
eq5 = (sum(u[i, j] for i in BE[j]) == x[j] for j in E if j not in S)
c5 = spptr_model.add_constraints(eq5, names='eq5')

# Eq.(6)
eq6 = (sum(x[i] for i in E) - sum(u[i, j] for i in E for j in FE[i]) == 1)
c6 = spptr_model.add_constraint(eq6, ctname='eq6')

In [231]:
# Define the objective function
obj = sum(x[i] * w[i] for i in E)
spptr_model.set_objective('min', obj)

In [232]:
# Solve the model
sol = spptr_model.solve()
# spptr_model.print_information()
# spptr_model.print_solution()
# print(sol.solve_details)

In [233]:
# Print the solution and the path
sol_x = sol.get_value_list(x)
sol_u = sol.get_value_dict(u)
ind = []
result = []
path = [s]
for i in range(ne):
    if sol_x[i] != 0:
        ind.append(i)
for i in ind:
    if i in S:
        result.append(i)
        ind.remove(i)
while ind:
    i = result[-1]
    for j in ind:
        if sol_u[(i, j)] == 1:
            result.append(j)
            ind.remove(j)
            break
for i in result:
    path.append(edges[i][1])
print(path)

[0, 2, 1, 2, 5]


The shortest path is 0-2-1-2-5, and the length is 4.